# Задача 7. Нейронные сети

## Задача

- [ ] Найти данные в виде изображений для задачи классификации.
- [ ] Реализовать классы, необходимые для построения сети со следующими слоями
	- FullyConnectedLayer
	- ReluLayer
	- FullyConnectedLayer
- [ ] Использовать CrossEntropyLoss и L2-регуляризацию.
- [ ] Обучить модель на тренировочных данных, подбирать параметры (особенно learning rate) на валидационной и оценить качество на тестовой. Анализировать графики train/val loss, проверять на каждом шаге корректность вычисления градиентов с помощью разностной оценки.
- [ ] * (**+2 балла**) Добавить Batch normalization.
- [ ] * (**+2 балла**) В качестве оптимизатор использовать один из: Momentum, RMSprop.
- [ ] * (**+1 балл**) Также реализовать оптимизатор Adam.

## Датасет

[Butterfly Image Classification](https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification) — содержит более 10 000 изображений 75 различных видов бабочек.

Каждое изображение имеет размер 224x224 пикселя. Класс каждого изображения хранится в файле Training_set.csv.

In [8]:
import kagglehub
import pandas as pd
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
)
from sklearn.cluster import AffinityPropagation
from statistics import mean
import seaborn as sns
import time
from itertools import product
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tensorflow as tf


In [9]:
folder = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")
training_set_path = os.path.join(folder, "Training_set.csv")
testing_set_path = os.path.join(folder, "Testing_set.csv")

training_df = pd.read_csv(training_set_path)
testing_df = pd.read_csv(testing_set_path)

training_df.head(5)

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART


In [10]:
class_counts = training_df['label'].value_counts().sort_values(ascending=False)

class_counts

label
MOURNING CLOAK    131
SLEEPY ORANGE     107
ATALA             100
BROWN SIPROETA     99
SCARCE SWALLOW     97
                 ... 
AMERICAN SNOOT     74
GOLD BANDED        73
MALACHITE          73
CRIMSON PATCH      72
WOOD SATYR         71
Name: count, Length: 75, dtype: int64

In [13]:
train_img_folder = os.path.join(folder, "train")


def load_image(filename):
    img_path = os.path.join(train_img_folder, filename)
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return img


training_df["image"] = training_df["filename"].apply(load_image)

training_df.head(3)

,filename,label,image
0,Image_1.jpg,SOUTHERN DOGFACE,"[[[0.09803922, 0.16862746, 0.12156863], [0.098..."
1,Image_2.jpg,ADONIS,"[[[0.57254905, 0.56078434, 0.36078432], [0.6, ..."
2,Image_3.jpg,BROWN SIPROETA,"[[[0.9254902, 0.9098039, 0.8980392], [0.925490..."
